

What's differentiate metric learning from traditional classification is that we train the model to learn a compact representation

how to have examples of the same classes to be close together and the one f


References
- Original paper:
FaceNet: A Unified Embedding for Face Recognition and Clustering
https://arxiv.org/abs/1503.03832 

- Mining strategies:
https://openaccess.thecvf.com/content_WACV_2020/papers/Xuan_Improved_Embeddings_with_Easy_Positive_Triplet_Mining_WACV_2020_paper.pdf 


In [1]:
 %load_ext autoreload
 %autoreload 2

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import  TensorBoard
# from tqdm.auto import tqdm
from time import  time
from tabulate import  tabulate

In [3]:
from tensorflow_similarity.utils import tf_cap_memory
from tensorflow_similarity.losses import TripletLoss
from tensorflow_similarity.layers import MetricEmbedding
from tensorflow_similarity.model import SimilarityModel

In [4]:
tf_cap_memory()
print(tf.__version__)

2.3.1


# preparing data

Note: Tensorflow similarity expect y_train to be the examples class as integer so there is no need for processing

In [ ]:
# FIXME sampler here and select only half of the class

In [5]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = tf.constant(x_train / 255.0)
x_test = tf.constant(x_test / 255.0)
print('x_train', x_train.shape)
print('y_train', y_train.shape)

x_train (60000, 28, 28)
y_train (60000,)


# model creation

In [6]:
def get_model():
    tf.keras.backend.clear_session()
    inputs = tf.keras.layers.Input(shape=(28,28))
    x = layers.Reshape((28,28,1))(inputs)
    x = layers.Conv2D(16, 3, activation='relu')(x)
    x = layers.Conv2D(32, 3, activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = MetricEmbedding(10)(x)
    # outputs2 = MetricEmbedding(10)(x)
    return SimilarityModel(inputs, outputs)
model = get_model()
model.summary()

Model: "similarity_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
reshape (Reshape)            (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        4640      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0         
_________________________________________________________________
dense (Dense)                (None, 128)          

# Compilation

In [8]:

distance = 'cosine' #@param ["cosine"]{allow-input: false}
positive_mining_strategy = 'hard' #@param ["easy", "hard"]{allow-input: false}
negative_mining_strategy = 'hard' #@param ["easy", "hard"]{allow-input: false}
triplet_loss = TripletLoss(distance=distance,
    positive_mining_strategy=positive_mining_strategy,
    negative_mining_strategy=negative_mining_strategy, name='tfs', margin=1)

callbacks = [TensorBoard(log_dir='logs/%s' % int(time()), update_freq='batch')]
callbacks  = []

model = get_model()
# model.compile(optimizer='adam', loss=triplet_loss)
history = model.compile(optimizer='adam', loss=triplet_loss)


EPOCHS = 1 #@param{Integer}
BATCH_SIZE = 8 #@param{Integer}
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=callbacks)

7500/7500 [==============================] - 12s 2ms/step - loss: 0.1379 - val_loss: 0.0623


# Training

In [ ]:
from tensorflow_similarity import metrics
semi = TripletLoss(distance=distance,
    positive_mining_strategy='hard',
    negative_mining_strategy='semi-hard')

In [ ]:
for i in range(1, len(x_train), BATCH_SIZE):
    s= i + BATCH_SIZE
    x = x_train[i:s]
    y = y_train[i:s]
    preds = model.predict(x)
    tla = tfa_loss_local(y, preds)
    tl = triplet_loss(y, preds)
    if tl != tla:
        break
print(tla, tl)

In [ ]:
from tqdm.auto import tqdm

In [ ]:
@tf.function
def test_loss():
    y = tf.random.uniform((BATCH_SIZE, ), 0, 10, dtype=tf.int32)
    preds = tf.random.uniform((BATCH_SIZE, 10), 0, 1)
    tl = triplet_loss(y, preds)
    tla = tfa_loss_local(y, preds)
    same = tf.cast(tl, tf.float16) != tf.cast(tla, tf.float16)
    if not same:
        print(tl)
        print(tla)
    return same, tl, tla, preds, y

In [ ]:
for i in tqdm(range(10000)):
    same, tla, tl, preds, y = test_loss()
    if not same:
        print(tla)
        print(tl)
        print(y)
        print(preds)
        break

In [ ]:
print(tla)
print(tl)

In [ ]:
print(float(tla), float(tl)) #, npl)

In [ ]:
print("tl", triplet_loss(y, preds))
print("tfa", tfa_loss_local(y, preds))

In [ ]:
print(preds)

In [ ]:
print(tfa.__version__)
tal = tfa_local(y, preds, distance_metric='angular')
ta = tfa.losses.triplet.triplet_hard_loss(y, preds, distance_metric='angular')
print(ta, tal)

In [ ]:
triplet_hard_loss_np(y, preds, 1.0, angular_distance_np)